In [1]:
import numpy as np
from astropy.table import Table, vstack

In [2]:
path = './../catalogs/'
VU_GC = Table.read(path+'VUGC_pPXF_2017_v2.fits')
VD_GC = Table.read(path+'VDGC_pPXF_2017_v2.fits')

In [3]:
columns_to_change= ['ZERR', 'ZERR_ne', 'ZERR_pe', 'ZHEL', 'ZOBS', 'ZOBS_ne', 'ZOBS_pe']

for col in columns_to_change:
    VD_GC[col] = VD_GC[col] * 3e+5
    
    VD_GC.rename_column(col,'V' + col[1:])
    
VU_GC['KECKID'] = '---'
VU_GC['ZQUAL'] = -101010
VU_GC['ZSPECNUM'] = -101010



In [4]:
test_VDGC = VD_GC[VD_GC['ZCONF']==1]  # cuts down 'ZCONF' to 1
test_VDGC = test_VDGC[test_VDGC['GCSAT']== 'Y'] # cuts down 'GCSAT'= yes
test_VUGC= VU_GC[VU_GC['ZCONF']==1]
test_VUGC= test_VUGC[test_VUGC['GCSAT']=='Y']
copy_VUGC= test_VUGC.copy()
Host = (test_VUGC['HOST']== 'VLSB-B') | (test_VUGC['HOST']== 'VLSB-D') | (test_VUGC['HOST']=='VCC0615')
test_VUGC = test_VUGC[Host]
not_host = ~Host
print(copy_VUGC[not_host])
test_VDGC = vstack([test_VDGC, copy_VUGC[not_host]])


VCC TARGTYPE GCSAT   HOST  ... KECKID  ZQUAL  ZSPECNUM
--- -------- ----- ------- ... ------ ------- --------
---     GCXL     Y VCC1148 ...    --- -101010  -101010
---     GCBL     Y VCC1153 ...    --- -101010  -101010


In [5]:
# test_VDGC.remove_rows(np.nonzero((test_VDGC['VHEL']>-300)  & (test_VDGC['VHEL'] < 300)))
#  Host.remove_rows(np.nonzero((Host['VHEL'] > -300) & (Host['VHEL'] < 300)))

In [6]:
VUGC_VDGC=vstack([test_VDGC, test_VUGC])

VUGC_VDGC

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,KECKID,MASKNAME,SLITNUM,ZSPECNUM,ZQUAL,YLOW,YHIGH,SPEC1DNAME
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str20,bytes20,int16,int64,int64,int16,int16,bytes40
---,GCA,Y,VCC1075,187.05533,10.299723,1795.8826,6.6943264,7.1255054,5.546625,1.0,1810.4825,7.014853,5.403738,0.00026271507,8.415822e-05,7.671974e-05,4.866667e-05,3.34,gcA10,vdgc1,11,12,3,25,31,spec1d.vdgc1.011.gcA10.fits.gz
---,GCA,Y,VCC1075,187.04483,10.286805,1843.0951,10.339908,9.879919,10.394689,1.0,1855.5137,6.974825,6.2042923,-7.271697e-06,2.2949209e-05,2.7485681e-05,4.866667e-05,5.36,gcA17,vdgc1,12,13,4,17,23,spec1d.vdgc1.012.gcA17.fits.gz
---,GCA,Y,VCC1075,187.03674,10.289222,1802.7073,6.052629,4.5716085,6.9834056,1.0,1821.8304,3.3122916,6.584555,1.50770975e-05,9.640003e-06,6.5378563e-06,4.866667e-05,7.06,gcA19,vdgc1,14,15,4,39,46,spec1d.vdgc1.014.gcA19.fits.gz
---,GCA,Y,VCC1075,187.04433,10.315306,1861.2278,2.5437276,1.4287839,1.2718638,1.0,1875.8278,0.69047767,0.23015922,0.0012339203,0.00022345458,0.00019736988,4.866667e-05,2.93,gcA20,vdgc1,15,16,1,32,36,spec1d.vdgc1.015.gcA20.fits.gz
---,GCN,Y,VCC1079,187.0423,10.3538885,1323.2156,12.560184,13.214163,11.683929,1.0,1333.0521,12.498647,9.956888,-1.5877651e-05,1.3676128e-05,1.9947132e-05,4.866667e-05,6.54,gcN120,vdgc1,26,31,1,122,128,spec1d.vdgc1.026.gcN120.fits.gz
---,GCN,Y,VCC1079,187.05162,10.367778,1338.6162,13.627155,13.83195,13.110909,1.0,1369.9778,9.316445,7.795393,5.5871988e-05,3.38234e-05,3.4890803e-05,4.866667e-05,6.97,gcN138,vdgc1,27,32,4,32,38,spec1d.vdgc1.027.gcN138.fits.gz
---,GCN,Y,VCC1079,187.05696,10.373972,1313.909,11.331903,11.525938,10.722818,1.0,1330.3203,7.775379,7.475171,6.03751e-06,2.805274e-05,2.5284158e-05,4.866667e-05,4.94,gcN145,vdgc1,29,34,4,20,26,spec1d.vdgc1.029.gcN145.fits.gz
---,GCN,Y,VCC1079,187.05429,10.377334,1358.58,6.8130116,8.182219,4.8669662,1.0,1376.2621,7.945497,4.2029076,1.0273774e-05,5.0034614e-06,7.0382025e-06,4.866667e-05,7.02,gcN147,vdgc1,30,35,4,30,36,spec1d.vdgc1.030.gcN147.fits.gz
---,GCN,Y,VCC1076,187.0515,10.524445,1650.4119,11.4403515,11.538051,10.929236,1.0,1669.485,10.947573,10.317138,1.4910315e-05,1.1407892e-05,1.12744665e-05,4.866667e-05,6.99,gcN206,vdgc1,37,44,4,114,120,spec1d.vdgc1.037.gcN206.fits.gz


In [7]:
fileOrphanDE = Table.read(path + 'orphanGCs.fits')
filekinDE = Table.read(path + 'VDGC_kinematic_prob.fits')
fileOrphanUDG = Table.read(path + 'SelectGCsTrue2.fits')
filekinUDG = Table.read(path + 'SelectGCsTrue_kinematic_prob.fits')

In [8]:
def typeandprob(file):
    probability= []
    objecttype= []
    for row in file:
        probability.append(max(row))
        vals = []
        for ind, val in enumerate(row):
            vals.append(val)
        if (vals.index(max(vals))==0):
            objecttype.append('Star')
        elif (vals.index(max(vals))==1):
            objecttype.append('ICGC')
        elif (vals.index(max(vals))==2):
            objecttype.append('M87GC')
    return probability, objecttype

In [9]:
probabilityDE, objecttypeDE = typeandprob(filekinDE)
probabilityUDG, objecttypeUDG = typeandprob(filekinUDG)


In [10]:
fileOrphanDE['Probability'] = probabilityDE
fileOrphanUDG['Probability'] = probabilityUDG
fileOrphanDE['Type'] = objecttypeDE
fileOrphanUDG['Type'] = objecttypeUDG

In [11]:
noDE = VD_GC[(VD_GC['ZCONF']==1) & (VD_GC['GCSAT']=='N') | (VD_GC['GCSAT']=='M')]
noUDG = VU_GC[(VU_GC['ZCONF']==1) & (VU_GC['GCSAT']=='N') | (VU_GC['GCSAT']=='M')]


In [12]:
noDE.sort('SPEC1DNAME')
noUDG.sort('SPEC1DNAME')
fileOrphanDE.sort('SPEC1DNAME')
fileOrphanUDG.sort('SPEC1DNAME')

focusedUDG = fileOrphanUDG['SPEC1DNAME', 'Probability', 'Type']
focusedUDG.add_row(['spec1d.vugc1.044.GCXO0504.fits.gz', -1, 'Unknown'])
focusedUDG.sort('SPEC1DNAME')

noDE['Probability'] = probabilityDE
noUDG['Probability'] = focusedUDG['Probability']
noDE['Type'] = objecttypeDE
noUDG['Type'] = focusedUDG['Type']

In [13]:
noUDG

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,MASKNAME,SLITNUM,YLOW,YHIGH,SPEC1DNAME,KECKID,ZQUAL,ZSPECNUM,Probability,Type
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes20,int16,int16,int16,bytes40,str3,int64,int64,float32,str5
VCC1148,NUC,N,---,187.2422,12.661695,1417.16,0.30438462,0.21,0.24,1.0,1403.28,0.21,0.24,-23.48,0.18,0.23,9.6,151.96,vugc1,0,43,59,spec1d.vugc1.000.NUC001.fits.gz,---,-101010,-101010,0.84855264,M87GC
---,GCXL,N,---,187.0645,12.741167,-591.31,5.6009865,3.6,4.99,1.0,-544.81,3.6,4.99,36.9,3.44,3.75,9.6,7.79,vugc1,7,58,68,spec1d.vugc1.007.GCXL008.fits.gz,---,-101010,-101010,0.9999366,ICGC
---,GCXL,N,---,187.04138,12.743,-339.2,12.014878,1.14,18.9,1.0,-271.02,1.14,18.9,58.58,5.66,7.6,9.6,2.14,vugc1,9,41,51,spec1d.vugc1.009.GCXL011.fits.gz,---,-101010,-101010,0.6393685,ICGC
---,GCXL,N,---,187.26938,12.675,1265.91,10.11425,9.15,6.84,1.0,1251.6,9.15,6.84,-23.91,6.78,5.61,9.6,4.19,vugc1,11,16,26,spec1d.vugc1.011.GCXL013.fits.gz,---,-101010,-101010,0.83778,M87GC
---,GCXL,N,---,187.0158,12.7225275,1397.37,7.2989125,5.45,7.26,1.0,1452.44,5.45,7.26,45.47,3.44,3.74,9.6,6.01,vugc1,12,12,22,spec1d.vugc1.012.GCXL015.fits.gz,---,-101010,-101010,0.8498808,M87GC
---,GCXL,N,---,187.25525,12.648389,951.12,30.016117,12.48,16.38,1.0,943.01,12.48,16.38,-17.71,28.32,24.32,9.6,2.21,vugc1,14,60,70,spec1d.vugc1.014.GCXL017.fits.gz,---,-101010,-101010,0.57101935,M87GC
---,GCBL,N,---,187.07175,12.729305,190.86,12.783288,3.2,4.53,1.0,220.64,3.2,4.53,20.18,13.53,10.84,9.6,7.97,vugc1,25,11,19,spec1d.vugc1.025.GCBL001.fits.gz,---,-101010,-101010,0.9511431,Star
---,GCBL,N,---,187.27963,12.6585,-14.56,0.59682494,0.53,0.49,1.0,-35.26,0.53,0.49,-30.3,0.3,0.32,9.6,61.11,vugc1,29,48,58,spec1d.vugc1.029.GCBL007.fits.gz,---,-101010,-101010,0.98372805,Star
---,GCBL,N,---,187.24742,12.637139,-55.26,5.19,4.92,5.46,1.0,-45.66,4.92,5.46,-8.24,29.95,48.57,9.6,7.79,vugc1,30,13,23,spec1d.vugc1.030.GCBL008.fits.gz,---,-101010,-101010,0.98111,Star


In [14]:
noDE_UDG=vstack([noDE,noUDG])

noDE_UDG

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,KECKID,MASKNAME,SLITNUM,ZSPECNUM,ZQUAL,YLOW,YHIGH,SPEC1DNAME,Probability,Type
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str20,bytes20,int16,int64,int64,int16,int16,bytes40,float32,str5
---,ALG,N,---,187.0097,10.359333,54.56785,2.5169947,1.2761084,1.2873586,1.0,117.61137,0.21014537,0.21014537,0.00016147838,4.6698975e-07,7.3384103e-07,4.866667e-05,196.03,alg42,vdgc1,0,0,-1,15,21,spec1d.vdgc1.000.alg42.fits.gz,0.98157275,Star
---,ALG,N,---,187.07608,10.3505,-210.7357,3.0194468,2.024616,2.1834822,1.0,-170.35785,1.5010384,1.6711562,8.592611e-05,1.7678897e-06,2.1348103e-06,4.866667e-05,58.89,alg115,vdgc1,1,1,-1,15,21,spec1d.vdgc1.001.alg115.fits.gz,0.90411496,Star
---,ALG,N,---,186.9995,10.459528,12.869002,2.8418825,1.7621847,1.9203647,1.0,76.86318,1.0807476,1.2108377,0.00016464724,2.0347409e-06,2.701869e-06,4.866667e-05,44.0,alg140,vdgc1,2,2,-1,15,21,spec1d.vdgc1.002.alg140.fits.gz,0.982906,Star
---,ALG,N,---,187.04004,10.45025,19.96391,2.5854344,1.3975698,1.426399,1.0,32.882748,0.46031845,0.57039464,-5.6038766e-06,1.4009692e-06,1.2675436e-06,4.866667e-05,80.71,alg164,vdgc1,3,3,-1,16,21,spec1d.vdgc1.003.alg164.fits.gz,0.9828415,Star
---,ALG,N,---,187.07025,10.325111,-2.7317898,2.612219,1.4655958,1.453245,1.0,34.784065,0.65045,0.6404431,7.6386175e-05,1.3342564e-06,1.2341872e-06,4.866667e-05,92.15,alg177,vdgc1,4,4,-1,15,20,spec1d.vdgc1.004.alg177.fits.gz,0.98282486,Star
---,ALG,N,---,187.05908,10.544111,17.422152,2.510731,1.271273,1.2664591,1.0,-29.680532,0.17011769,0.14009692,-0.00020567562,5.003461e-07,4.6698975e-07,4.866667e-05,222.86,alg191,vdgc1,5,5,-1,15,21,spec1d.vdgc1.005.alg191.fits.gz,0.9828719,Star
---,ALG,N,---,187.06075,10.386139,-53.867165,2.7268543,1.7217433,1.5956256,1.0,-18.522814,0.73050535,0.70048463,6.9147834e-05,3.1021461e-06,2.3349487e-06,4.866667e-05,73.2,alg194,vdgc1,6,6,-1,15,20,spec1d.vdgc1.006.alg194.fits.gz,0.9802294,Star
VCC1075,NUC,N,---,187.05128,10.297639,1811.7435,2.917756,1.9805114,1.9400272,1.0,1825.7831,1.4409969,1.2608724,-1.867959e-06,1.7678897e-06,2.6018e-06,4.866667e-05,28.58,nuc5,vdgc1,7,7,4,30,37,spec1d.vdgc1.007.nuc5.fits.gz,1.0,ICGC
VCC1079,NUC,N,---,187.05008,10.365334,1334.1931,3.0186136,2.089962,2.114991,1.0,1349.0332,1.5510731,1.541066,8.0055383e-07,2.101454e-06,2.435018e-06,4.866667e-05,32.52,nuc6,vdgc1,8,8,4,19,25,spec1d.vdgc1.008.nuc6.fits.gz,1.0,ICGC


In [15]:
table1=vstack([VUGC_VDGC,noDE_UDG])

table1

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,KECKID,MASKNAME,SLITNUM,ZSPECNUM,ZQUAL,YLOW,YHIGH,SPEC1DNAME,Probability,Type
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str20,bytes20,int16,int64,int64,int16,int16,bytes40,float32,str5
---,GCA,Y,VCC1075,187.05533,10.299723,1795.8826,6.6943264,7.1255054,5.546625,1.0,1810.4825,7.014853,5.403738,0.00026271507,8.415822e-05,7.671974e-05,4.866667e-05,3.34,gcA10,vdgc1,11,12,3,25,31,spec1d.vdgc1.011.gcA10.fits.gz,--,--
---,GCA,Y,VCC1075,187.04483,10.286805,1843.0951,10.339908,9.879919,10.394689,1.0,1855.5137,6.974825,6.2042923,-7.271697e-06,2.2949209e-05,2.7485681e-05,4.866667e-05,5.36,gcA17,vdgc1,12,13,4,17,23,spec1d.vdgc1.012.gcA17.fits.gz,--,--
---,GCA,Y,VCC1075,187.03674,10.289222,1802.7073,6.052629,4.5716085,6.9834056,1.0,1821.8304,3.3122916,6.584555,1.50770975e-05,9.640003e-06,6.5378563e-06,4.866667e-05,7.06,gcA19,vdgc1,14,15,4,39,46,spec1d.vdgc1.014.gcA19.fits.gz,--,--
---,GCA,Y,VCC1075,187.04433,10.315306,1861.2278,2.5437276,1.4287839,1.2718638,1.0,1875.8278,0.69047767,0.23015922,0.0012339203,0.00022345458,0.00019736988,4.866667e-05,2.93,gcA20,vdgc1,15,16,1,32,36,spec1d.vdgc1.015.gcA20.fits.gz,--,--
---,GCN,Y,VCC1079,187.0423,10.3538885,1323.2156,12.560184,13.214163,11.683929,1.0,1333.0521,12.498647,9.956888,-1.5877651e-05,1.3676128e-05,1.9947132e-05,4.866667e-05,6.54,gcN120,vdgc1,26,31,1,122,128,spec1d.vdgc1.026.gcN120.fits.gz,--,--
---,GCN,Y,VCC1079,187.05162,10.367778,1338.6162,13.627155,13.83195,13.110909,1.0,1369.9778,9.316445,7.795393,5.5871988e-05,3.38234e-05,3.4890803e-05,4.866667e-05,6.97,gcN138,vdgc1,27,32,4,32,38,spec1d.vdgc1.027.gcN138.fits.gz,--,--
---,GCN,Y,VCC1079,187.05696,10.373972,1313.909,11.331903,11.525938,10.722818,1.0,1330.3203,7.775379,7.475171,6.03751e-06,2.805274e-05,2.5284158e-05,4.866667e-05,4.94,gcN145,vdgc1,29,34,4,20,26,spec1d.vdgc1.029.gcN145.fits.gz,--,--
---,GCN,Y,VCC1079,187.05429,10.377334,1358.58,6.8130116,8.182219,4.8669662,1.0,1376.2621,7.945497,4.2029076,1.0273774e-05,5.0034614e-06,7.0382025e-06,4.866667e-05,7.02,gcN147,vdgc1,30,35,4,30,36,spec1d.vdgc1.030.gcN147.fits.gz,--,--
---,GCN,Y,VCC1076,187.0515,10.524445,1650.4119,11.4403515,11.538051,10.929236,1.0,1669.485,10.947573,10.317138,1.4910315e-05,1.1407892e-05,1.12744665e-05,4.866667e-05,6.99,gcN206,vdgc1,37,44,4,114,120,spec1d.vdgc1.037.gcN206.fits.gz,--,--


In [16]:
from astropy.io import fits,ascii

NucDat = ascii.read(path + 'nuc.dat')

NucDat

name,div1,g_n_bestfit,g_n_bestfit_error,g_Ie_cg,g_re_cg,g_mag_cg,g_avmure_cg,g_I0_cg,div2,g_Ie_bestfit,g_Ie_bestfit_error,g_re_bestfit,g_re_bestfit_error,g_mag_bestfit,g_mag_bestfit_error,g_avmure_bestfit,g_avmure_bestfit_error,div3,g_Ie_inner,g_Ie_inner_error,g_n_inner,g_n_inner_error,g_re_inner,g_re_inner_error,g_mag_inner,g_mag_inner_error,g_avmure_inner,g_avmure_inner_error,div4,g_Ie_outer,g_Ie_outer_error,g_n_outer,g_n_outer_error,g_re_outer,g_re_outer_error,g_mag_outer,g_mag_outer_error,g_avmure_outer,g_avmure_outer_error,div5,g_concentration,g_sky,div6,g_av_ell,g_av_ell_error,g_av_pa,g_av_pa_error,g_av_A3,g_av_A3_error,g_av_A4,g_av_A4_error,g_av_B3,g_av_B3_error,g_av_B4,g_av_B4_error,g_PSF_FWHM,div7,u_n_bestfit,u_n_bestfit_error,u_Ie_cg,u_re_cg,u_mag_cg,u_avmure_cg,u_I0_cg,div8,u_Ie_bestfit,u_Ie_bestfit_error,u_re_bestfit,u_re_bestfit_error,u_mag_bestfit,u_mag_bestfit_error,u_avmure_bestfit,u_avmure_bestfit_error,div9,u_Ie_inner,u_Ie_inner_error,u_n_inner,u_n_inner_error,u_re_inner,u_re_inner_error,u_mag_inner,u_mag_inner_error,u_avmure_inner,u_avmure_inner_error,div10,u_Ie_outer,u_Ie_outer_error,u_n_outer,u_n_outer_error,u_re_outer,u_re_outer_error,u_mag_outer,u_mag_outer_error,u_avmure_outer,u_avmure_outer_error,div11,u_concentration,u_sky,div12,u_av_ell,u_av_ell_error,u_av_pa,u_av_pa_error,u_av_A3,u_av_A3_error,u_av_A4,u_av_A4_error,u_av_B3,u_av_B3_error,u_av_B4,u_av_B4_error,u_PSF_FWHM,div13,r_n_bestfit,r_n_bestfit_error,r_Ie_cg,r_re_cg,r_mag_cg,r_avmure_cg,r_I0_cg,div14,r_Ie_bestfit,r_Ie_bestfit_error,r_re_bestfit,r_re_bestfit_error,r_mag_bestfit,r_mag_bestfit_error,r_avmure_bestfit,r_avmure_bestfit_error,div15,r_Ie_inner,r_Ie_inner_error,r_n_inner,r_n_inner_error,r_re_inner,r_re_inner_error,r_mag_inner,r_mag_inner_error,r_avmure_inner,r_avmure_inner_error,div16,r_Ie_outer,r_Ie_outer_error,r_n_outer,r_n_outer_error,r_re_outer,r_re_outer_error,r_mag_outer,r_mag_outer_error,r_avmure_outer,r_avmure_outer_error,div17,r_concentration,r_sky,div18,r_av_ell,r_av_ell_error,r_av_pa,r_av_pa_error,r_av_A3,r_av_A3_error,r_av_A4,r_av_A4_error,r_av_B3,r_av_B3_error,r_av_B4,r_av_B4_error,r_PSF_FWHM,div19,i_n_bestfit,i_n_bestfit_error,i_Ie_cg,i_re_cg,i_mag_cg,i_avmure_cg,i_I0_cg,div20,i_Ie_bestfit,i_Ie_bestfit_error,i_re_bestfit,i_re_bestfit_error,i_mag_bestfit,i_mag_bestfit_error,i_avmure_bestfit,i_avmure_bestfit_error,div21,i_Ie_inner,i_Ie_inner_error,i_n_inner,i_n_inner_error,i_re_inner,i_re_inner_error,i_mag_inner,i_mag_inner_error,i_avmure_inner,i_avmure_inner_error,div22,i_Ie_outer,i_Ie_outer_error,i_n_outer,i_n_outer_error,i_re_outer,i_re_outer_error,i_mag_outer,i_mag_outer_error,i_avmure_outer,i_avmure_outer_error,div23,i_concentration,i_sky,div24,i_av_ell,i_av_ell_error,i_av_pa,i_av_pa_error,i_av_A3,i_av_A3_error,i_av_A4,i_av_A4_error,i_av_B3,i_av_B3_error,i_av_B4,i_av_B4_error,i_PSF_FWHM,div25,z_n_bestfit,z_n_bestfit_error,z_Ie_cg,z_re_cg,z_mag_cg,z_avmure_cg,z_I0_cg,div26,z_Ie_bestfit,z_Ie_bestfit_error,z_re_bestfit,z_re_bestfit_error,z_mag_bestfit,z_mag_bestfit_error,z_avmure_bestfit,z_avmure_bestfit_error,div27,z_Ie_inner,z_Ie_inner_error,z_n_inner,z_n_inner_error,z_re_inner,z_re_inner_error,z_mag_inner,z_mag_inner_error,z_avmure_inner,z_avmure_inner_error,div28,z_Ie_outer,z_Ie_outer_error,z_n_outer,z_n_outer_error,z_re_outer,z_re_outer_error,z_mag_outer,z_mag_outer_error,z_avmure_outer,z_avmure_outer_error,div29,z_concentration,z_sky,div30,z_av_ell,z_av_ell_error,z_av_pa,z_av_pa_error,z_av_A3,z_av_A3_error,z_av_A4,z_av_A4_error,z_av_B3,z_av_B3_error,z_av_B4,z_av_B4_error,z_PSF_FWHM,div31,U(REFF),G(REFF),R(REFF),I(REFF),Z(REFF),div32,g_mu(1arcsec),g_mu(2arcsec),g_mu(3arcsec),g_mu(4arcsec),g_mu(5arcsec),div33,u_mu(1arcsec),u_mu(2arcsec),u_mu(3arcsec),u_mu(4arcsec),u_mu(5arcsec),div34,r_mu(1arcsec),r_mu(2arcsec),r_mu(3arcsec),r_mu(4arcsec),r_mu(5arcsec),div35,i_mu(1arcsec),i_mu(2arcsec),i_mu(3arcsec),i_mu(4arcsec),i_mu(5arcsec),div36,z_mu(1arcsec),z_mu(2arcsec),z_mu(3arcsec),z_mu(4arcsec),z_mu(5arcsec),div37,E(B-V),RA_de

In [17]:
NucDat.sort('VCC')
table1.sort('VCC')
print(table1['VCC'])
print(NucDat['VCC'])
print(len(NucDat[0]))

  VCC  
-------
    ---
    ---
    ---
    ---
    ---
    ---
    ---
    ---
    ---
    ---
    ...
VCC1539
VCC1545
VCC1561
VCC1563
VCC1809
VCC1828
VCC1861
VCC1870
VCC1871
VCC1879
 VLSB-D
Length = 547 rows
VCC 
----
1075
1076
1079
1396
1407
1420
1488
1491
1499
1509
1523
1539
1545
1561
1563
1809
1828
1861
1870
1871
1879
335


In [34]:
nuc_leus = list(NucDat['Nucleus'])
nada = []
for i in range(len(table1)):
    nada.append('0')
    
table1['Nucleus'] = nada

In [35]:
table1

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,KECKID,MASKNAME,SLITNUM,ZSPECNUM,ZQUAL,YLOW,YHIGH,SPEC1DNAME,Probability,Type,Nucleus
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str20,bytes20,int16,int64,int64,int16,int16,bytes40,float32,str5,str1
---,GCA,Y,VCC1075,187.05533,10.299723,1795.8826,6.6943264,7.1255054,5.546625,1.0,1810.4825,7.014853,5.403738,0.00026271507,8.415822e-05,7.671974e-05,4.866667e-05,3.34,gcA10,vdgc1,11,12,3,25,31,spec1d.vdgc1.011.gcA10.fits.gz,--,--,0
---,GCN,N,---,190.08891,12.870111,78.57377,7.9413695,6.3965297,8.894488,1.0,101.420166,0.33022848,0.0,3.0821324e-05,2.0881113e-05,2.935364e-05,4.5333334e-05,5.76,gcN110,vdgc7,22,23,3,5,11,spec1d.vdgc7.022.gcN110.fits.gz,0.97961456,Star,0
---,GCN,N,---,190.07875,12.867028,49.66377,2.5301738,1.3076181,1.3063538,1.0,41.94902,0.35024232,0.36024922,-7.104915e-05,5.003461e-07,3.6692052e-07,4.5333334e-05,117.14,gcN106,vdgc7,21,22,4,53,60,spec1d.vdgc7.021.gcN106.fits.gz,0.9818553,Star,0
---,GCN,N,---,190.10533,12.837694,25.286901,11.844055,13.983142,9.384307,1.0,34.123608,13.23916,8.255712,-1.5877651e-05,1.4409969e-05,1.4276543e-05,4.5333334e-05,6.79,gcN88,vdgc7,20,21,4,72,78,spec1d.vdgc7.020.gcN88.fits.gz,0.98275113,Star,0
---,GCN,N,---,190.12062,12.822473,52.495724,6.1582446,6.7694,4.771162,1.0,58.780663,6.584555,4.5231295,-2.4383535e-05,3.168859e-06,2.868651e-06,4.5333334e-05,22.43,gcN79,vdgc7,18,19,4,14,19,spec1d.vdgc7.018.gcN79.fits.gz,0.9816966,Star,0
---,GCN,N,---,190.09659,12.819139,128.22812,2.5874429,1.3875024,1.4435312,1.0,141.82811,0.60041535,0.72049844,3.6692052e-07,8.439171e-06,0.00039550694,4.5333334e-05,16.6,gcN77,vdgc7,17,17,1,244,252,spec1d.vdgc7.017.gcN77.fits.gz,0.9713231,Star,0
---,GCA,N,---,190.05559,12.870611,-57.02004,15.824809,12.889712,18.465162,1.0,-43.42004,12.828875,18.422745,-0.0001186821,4.8700356e-05,0.00013362577,4.5333334e-05,2.23,gcA20,vdgc7,14,14,2,13,19,spec1d.vdgc7.014.gcA20.fits.gz,0.9799319,Star,0
---,GCA,N,---,190.05446,12.867416,611.40234,43.547455,41.135006,45.85438,1.0,611.50305,36.185036,40.64812,-4.4997796e-05,6.5078355e-05,7.061552e-05,4.5333334e-05,2.26,gcA19,vdgc7,13,13,2,23,29,spec1d.vdgc7.013.gcA19.fits.gz,0.9999256,ICGC,0
---,GCA,N,---,190.06741,12.847139,33.87284,5.809734,5.6177087,5.1643186,1.0,22.915854,5.453773,4.9834476,-8.1856626e-05,1.6678205e-06,1.7345333e-06,4.5333334e-05,29.5,gcA15,vdgc7,11,11,4,27,33,spec1d.vdgc7.011.gcA15.fits.gz,0.9825279,Star,0


In [36]:
miVCC= []
nucVCC = []
for i in range (len(table1)):
    if(table1[i]['VCC']== '---'):
        pass
    else:
        miVCC.append(table1[i]['VCC'])
for i in range(len(NucDat)):
    nucVCC.append(''+ (str(NucDat[i]['name'][24:])))
    